In [1]:
from haystack.telemetry import tutorial_running
tutorial_running(35)

In [2]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset('vblagoje/PubMedQA_instruction', split='train')
dataset = dataset.select(range(1000))

all_documents = [Document(content=doc['context']) for doc in dataset]
all_questions = [doc['instruction'] for doc in dataset]
all_ground_truth_answers = [doc['response'] for doc in dataset]

In [3]:
from typing import List
from haystack import Pipeline
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.document_stores.types import DuplicatePolicy

document_store = InMemoryDocumentStore()

document_embedder = SentenceTransformersDocumentEmbedder(model='sentence-transformers/all-MiniLM-L6-v2')
document_writer = DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE)

indexing = Pipeline()
indexing.add_component(instance=document_embedder, name='document_embedder')
indexing.add_component(instance=document_writer, name='document_writer')

indexing.connect('document_embedder.documents', 'document_writer.documents')

indexing.run({'document_embedder': {'documents': all_documents}})


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 1000}}

In [4]:
import os
from getpass import getpass
from haystack.components.builders import AnswerBuilder, PromptBuilder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators import OpenAIGenerator
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

template = """
You have to answer the following question based on the given context information only.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

rag_pipeline = Pipeline()
rag_pipeline.add_component("query_embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
rag_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store, top_k=3))
rag_pipeline.add_component("prompt_builder", PromptBuilder(template=template))
rag_pipeline.add_component("generator", OpenAIGenerator(model="gpt-4o-mini"))
rag_pipeline.add_component("answer_builder", AnswerBuilder())

rag_pipeline.connect("query_embedder", "retriever.query_embedding")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "generator")
rag_pipeline.connect("generator.replies", "answer_builder.replies")
rag_pipeline.connect("generator.meta", "answer_builder.meta")
rag_pipeline.connect("retriever", "answer_builder.documents")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - generator: OpenAIGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> answer_builder.documents (List[Document])
  - prompt_builder.prompt -> generator.prompt (str)
  - generator.replies -> answer_builder.replies (List[str])
  - generator.meta -> answer_builder.meta (List[Dict[str, Any]])

In [5]:
question = "Do high levels of procalcitonin in the early phase after pediatric liver transplantation indicate poor postoperative outcome?"

response = rag_pipeline.run(
    {
        "query_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "answer_builder": {"query": question},
    }
)
response["answer_builder"]["answers"][0].data

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Yes, high levels of procalcitonin (PCT) in the early phase after pediatric liver transplantation indicate a poor postoperative outcome. Patients with high PCT levels on postoperative day 2 experienced higher International Normalized Ratio values on postoperative day 5, suffered more frequently from primary graft non-function, had longer stays in the pediatric intensive care unit, and required more time on mechanical ventilation.'

In [6]:
import random

questions, ground_truth_answers, ground_truth_docs = zip(
  *random.sample(list(zip(all_questions, all_ground_truth_answers, all_documents)), 25)
)

In [8]:
rag_answers = []
retrieved_docs = []

for question in list(questions):
  response = rag_pipeline.run(
    {
      'query_embedder': {'text': question},
      'prompt_builder': {'question': question},
      'answer_builder': {'query': question}
    }
  )

  print(f"Question: {question}")  
  print("Answer from pipeline:")
  print(response["answer_builder"]["answers"][0].data)
  print("\n-----------------------------------\n")
  rag_answers.append(response["answer_builder"]["answers"][0].data)
  retrieved_docs.append(response["answer_builder"]["answers"][0].documents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Are pre-transplant impedance measures of reflux associated with early allograft injury after lung transplantation?
Answer from pipeline:
Yes, pre-transplant impedance measures of reflux are associated with early allograft injury after lung transplantation. Specifically, prolonged bolus clearance, increased total distal reflux episodes, and increased total proximal reflux episodes were significantly linked to decreased time to early allograft injury.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Does protein phosphatase 5 mediate corticosteroid insensitivity in airway smooth muscle in patients with severe asthma?
Answer from pipeline:
Yes, protein phosphatase 5 (PP5) mediates corticosteroid insensitivity in airway smooth muscle (ASM) in patients with severe asthma. Increased levels of PP5 in ASM cells from severe asthmatics were associated with reduced sensitivity to corticosteroids, and knockdown of PP5 restored the repressive action of fluticasone on chemokine production as well as its ability to induce GRα nuclear translocation and GRE-dependent GILZ expression.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Question: Is altered perineal microbiome associated with vulvovaginitis and urinary tract infection in preadolescent girls?
Answer from pipeline:
Yes, altered perineal microbiome is associated with vulvovaginitis and urinary tract infections (UTIs) in preadolescent girls. The presence of vulvovaginitis significantly increases periurethral colonization with uropathogenic bacteria, which in turn is linked to a higher prevalence of UTIs in girls with vulvovaginitis compared to those without.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Does leukocyte concentration and composition in platelet-rich plasma ( PRP ) influence the growth factor and protease concentrations?
Answer from pipeline:
Yes, the leukocyte concentration and composition in platelet-rich plasma (PRP) influence the growth factor and protease concentrations. The study found that the leukocyte concentration positively correlated with platelet-derived growth factor-BB (PDGF-BB) and vascular endothelial growth factor (VEGF) concentrations, while it negatively correlated with fibroblast growth factor-basic (FGF-b). Additionally, the concentration of matrix metalloproteinase-9 (MMP-9) strongly correlated with the leukocyte concentration.

-----------------------------------



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question: Is progression of EGFR mutant lung adenocarcinoma driven by alveolar macrophages?
Answer from pipeline:
Yes, the progression of EGFR mutant lung adenocarcinoma is driven by alveolar macrophages. The study indicates that the progression and remission of the tumor are dependent on the expansion and contraction of alveolar macrophages, with local proliferation being the mechanism underlying macrophage expansion. Additionally, the depletion of alveolar macrophages in tumor-bearing mice resulted in a reduction of tumor burden, highlighting their critical role in the development of EGFR mutant adenocarcinoma.

-----------------------------------



In [9]:
from haystack.components.evaluators.document_mrr import DocumentMRREvaluator
from haystack.components.evaluators.faithfulness import FaithfulnessEvaluator
from haystack.components.evaluators.sas_evaluator import SASEvaluator

eval_pipeline = Pipeline()
eval_pipeline.add_component("doc_mrr_evaluator", DocumentMRREvaluator())
eval_pipeline.add_component("faithfulness", FaithfulnessEvaluator())
eval_pipeline.add_component("sas_evaluator", SASEvaluator(model="sentence-transformers/all-MiniLM-L6-v2"))

results = eval_pipeline.run(
    {
        "doc_mrr_evaluator": {
            "ground_truth_documents": list([d] for d in ground_truth_docs),
            "retrieved_documents": retrieved_docs,
        },
        "faithfulness": {
            "questions": list(questions),
            "contexts": list([d.content] for d in ground_truth_docs),
            "predicted_answers": rag_answers,
        },
        "sas_evaluator": {"predicted_answers": rag_answers, "ground_truth_answers": list(ground_truth_answers)},
    }
)

d:\Projects\VirtualTA\venv\Lib\site-packages\transformers\models\auto\configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
d:\Projects\VirtualTA\venv\Lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(
100%|██████████| 25/25 [01:00<00:00,  2.44s/it]


In [10]:
from haystack.evaluation.eval_run_result import EvaluationRunResult

inputs = {
    "question": list(questions),
    "contexts": list([d.content] for d in ground_truth_docs),
    "answer": list(ground_truth_answers),
    "predicted_answer": rag_answers,
}

evaluation_result = EvaluationRunResult(run_name="pubmed_rag_pipeline", inputs=inputs, results=results)
evaluation_result.score_report()

,metrics,score
0,doc_mrr_evaluator,1.000000
1,faithfulness,1.000000
2,sas_evaluator,0.708761


In [11]:
import pandas as pd

results_df = evaluation_result.to_pandas()
top_3 = results_df.nlargest(3, "sas_evaluator")
bottom_3 = results_df.nsmallest(3, "sas_evaluator")
pd.concat([top_3, bottom_3])

,question,contexts,answer,predicted_answer,doc_mrr_evaluator,faithfulness,sas_evaluator
11,Do central adenosine A1 and A2A receptors medi...,[The present study aimed to investigate the in...,The above findings suggest that: (i) NPS evoke...,"Yes, central adenosine A1 and A2A receptors me...",1.0,1.0,0.891792
15,Does nT-proBNP and Echocardiographic Parameter...,[Natriuretic peptides and echocardiographic pa...,Our data confirm NT-proBNP is an independent p...,"Yes, both nT-proBNP and echocardiographic para...",1.0,1.0,0.866158
0,Are pre-transplant impedance measures of reflu...,[Acid reflux has been associated with poorer o...,"Prolonged bolus clearance, increased total dis...","Yes, pre-transplant impedance measures of refl...",1.0,1.0,0.855975
6,Are multiple pathways responsible for anti-inf...,[Hordeum vulgare L. (HV or barley) is used by ...,These results suggest the likely mechanisms re...,"Yes, multiple pathways are responsible for the...",1.0,1.0,0.401589
23,Does leukocyte concentration and composition i...,[Platelet-rich plasma (PRP) therapy has become...,These findings demonstrate that leukocytes str...,"Yes, the leukocyte concentration and compositi...",1.0,1.0,0.529442
13,Does early-life stress selectively affect gast...,[Early-life stress and a genetic predispositio...,"Our data suggest that early-life stress, on th...","Yes, early-life stress selectively affects gas...",1.0,1.0,0.598352
